# Azure AI Search

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import AzureAISearchQueryType, AzureAISearchTool, ListSortOrder, MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [3]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup the Azure AI Search tool

Azure AI Search Query Types: https://learn.microsoft.com/en-us/python/api/azure-ai-agents/azure.ai.agents.models.azureaisearchquerytype?view=azure-python-preview

In [4]:
# Enter Azure AI Search details here
search_connection = "dlt4search"  # Replace with your Azure AI Search connection name
search_index ="healthplanindex" # Replace with your Azure AI Search index name

# Extract the connection list.
conn_list = project_client.connections.list()
search_connection_id = ""

# Find the connection ID for Azure AI Search
for conn in conn_list:
    if conn.name == search_connection:
        search_connection_id = conn.id


# Print the connection names
print(f"Search Connection ID: {search_connection_id}")

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(
    index_connection_id=search_connection_id,
    index_name=search_index,
    query_type=AzureAISearchQueryType.VECTOR_SEMANTIC_HYBRID,
    top_k=5,
    filter="",
)


Search Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/agent-standard/providers/Microsoft.CognitiveServices/accounts/foundrydlt4/projects/projectdlt4/connections/dlt4search


## Create the agent

In [5]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-aisearch-agent",  # Name of the agent
    instructions="You are a helpful agent and can search information from uploaded files",  # Instructions for the agent
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_x44I8MamgtVZjZ4qtHh4gfJK


## Create thread

In [6]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_MnIOGGfftNsF5Fpjnv695utA


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [7]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        # Fetch the first message in the thread to see the agent's response
        messages = list(project_client.agents.messages.list(thread_id=thread.id))
        if messages:
            first_message = messages[0]
            content = first_message['content']
            # Extract the text value if present
            if isinstance(content, list) and content:
                text_value = content[0].get('text', {}).get('value', '')
                print(text_value)

In [8]:
def run_agent_details(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Fetch run steps to get the details of the agent run
    run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                azure_ai_search_details = call.get("azure_ai_search", {})
                if azure_ai_search_details:
                    print(f"    azure_ai_search input: {azure_ai_search_details.get('input')}")
                    print(f"    azure_ai_search output: {azure_ai_search_details.get('output')}")
        print()  # add an extra newline between steps

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        for message in messages:
            if message.role == MessageRole.AGENT and message.url_citation_annotations:
                placeholder_annotations = {
                    annotation.text: f" [see {annotation.url_citation.title}] ({annotation.url_citation.url})"
                    for annotation in message.url_citation_annotations
                }
                for message_text in message.text_messages:
                    message_str = message_text.text.value
                    for k, v in placeholder_annotations.items():
                        message_str = message_str.replace(k, v)
                    print(f"{message.role}: {message_str}")
            else:
                for message_text in message.text_messages:
                    print(f"{message.role}: {message_text.text.value}")

## Run the agent

In [9]:
user_input = "Provide details on the Northwind Standard Plan"
run_agent(user_input)

Created message, ID: msg_eQR61Act343rW3sEVsCtVb43
Run finished with status: completed
The Northwind Standard Plan is a comprehensive health plan that provides coverage for medical, vision, and dental services. It includes preventive care services and prescription drug coverage, allowing you to choose from various in-network providers, such as primary care physicians, specialists, hospitals, and pharmacies【3:0†source】【3:2†source】.

### Summary of Coverage

1. **Covered Services:**
    - Preventive care services (physical exams, immunizations, screenings)
    - Diagnostic tests and treatments
    - Medical consultations
    - Physical therapy
    - Mental health services
    - Prescription drugs
    - Inpatient hospital services
    - Emergency services【3:1†source】

2. **Exclusions:**
    - Cosmetic services
    - Experimental treatments
    - Most dental services
    - Services provided by out-of-network providers
    - Emergency services, mental health, and substance abuse services【3:0

In [10]:
user_input = "Provide a tabular comparison between the Northwind Standard and Northwind Healthplus plans"
run_agent_details(user_input)

Created message, ID: msg_yDuZilJndb3PiZ0D2lTmiPef
Run finished with status: completed
Step step_lWi8u5UFD7HZkp840Wru4lJx status: completed

Step step_HgvvPClePocDE5cQlMSOUA89 status: completed
  Tool calls:
    Tool Call ID: call_ALybMRiKA1D4liuRSKO3nvK2
    Type: azure_ai_search
    azure_ai_search input: Northwind Standard vs Healthplus Plan comparison
    azure_ai_search output: {'summary': 'Retrieved 5 documents.', 'metadata': {'urls': ['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4'], 'titles': ['Northwind_Health_Plus_Benefits_Details.pdf', 'Northwind_Health_Plus_Benefits_Details.pdf', 'Northwind_Health_Plus_Benefits_Details.pdf', 'Northwind_Health_Plus_Benefits_Details.pdf', 'Northwind_Health_Plus_Benefits_Details.pdf'], 'get_urls': ['https://dlt4search.search.windows.net/indexes/healthplanindex/docs/051d519df6ff_aHR0cHM6Ly9kbHQ0c3RvcmFnZS5ibG9iLmNvcmUud2luZG93cy5uZXQvaGVhbHRocGxhbnMvTm9ydGh3aW5kX0hlYWx0aF9QbHVzX0JlbmVmaXRzX0RldGFpbHMucGRm0_pages_1?api-version=2024-07-01&$select=chun

## Cleanup resources

In [36]:
# Cleanup resources

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted agent


## Azure AI Search vs File Search
- Azure AI Search can schedule index changes
- More flexiblity
  - embedding model / size
  - field composition
  - add skills